# Yelp Data Challenge - Data Preprocessing

BitTiger DS501

Zhenning Tan, 6/16/2017

## Dataset Introduction

[Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge)

The Challenge Dataset:

    4.1M reviews and 947K tips by 1M users for 144K businesses
    1.1M business attributes, e.g., hours, parking availability, ambience.
    Aggregated check-ins over time for each of the 125K businesses
    200,000 pictures from the included businesses

Cities:

    U.K.: Edinburgh
    Germany: Karlsruhe
    Canada: Montreal and Waterloo
    U.S.: Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, Cleveland

Files:

    yelp_academic_dataset_business.json
    yelp_academic_dataset_checkin.json
    yelp_academic_dataset_review.json
    yelp_academic_dataset_tip.json
    yelp_academic_dataset_user.json

Notes on the Dataset

    Each file is composed of a single object type, one json-object per-line.
    Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.



## Read data from file and load to Pandas DataFrame

**Warning**: Loading all the 1.8 GB data into Pandas at a time takes long time and a lot of memory!

In [1]:
import json
import numpy as np
import pandas as pd
from time import time

In [2]:
file_business, file_checkin, file_review, file_tip, file_user = [
    'yelp_dataset_challenge_round9/yelp_academic_dataset_business.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_checkin.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_review.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_tip.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_user.json'
]

#### Business Data

In [ ]:
with open(file_business) as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
df_business.head(2)

In [ ]:
df_business.info()

#### Checkin Data

In [ ]:
#with open(file_checkin) as f:
#     df_checkin = pd.DataFrame(json.loads(line) for line in f)
#df_checkin.head(2)

#### Review Data

In [ ]:
#with open(file_review) as f:
#     df_review = pd.DataFrame(json.loads(line) for line in f)
#df_review.head(2)

#### Tip Data

In [ ]:
#with open(file_tip) as f:
#     df_tip = pd.DataFrame(json.loads(line) for line in f)
#df_tip.head(2)

#### User Data

In [ ]:
# with open(file_user) as f:
#     df_user = pd.DataFrame(json.loads(line) for line in f)
# df_user.head(2)

## Filter data by city and category

#### Create filters/masks

* create filters that selects business 
    * that are located in "Las Vegas"
    * that contains "Restaurants" in their category (You may need to filter null categories first)

In [ ]:
# Create Pandas DataFrame filters
df_filtered = df_business[df_business['city'] == "Las Vegas"]

In [ ]:
restaurant_filter = (~df_business["categories"].isnull()) & (df_business["categories"][:10]).apply(lambda x: "Restaurants" in x)

In [ ]:
#remove null categories
df_filtered = df_filtered[~df_filtered["categories"].isnull()]

In [ ]:
# find categories with restaurants
df_filtered = df_filtered[df_filtered["categories"].apply(lambda x: "Restaurants" in x)]

In [ ]:
df_filtered.shape

#### Keep relevant columns

* only keep some useful columns
    * business_id
    * name
    * categories
    * stars

In [ ]:
selected_features = [u'business_id', u'name', u'categories', u'stars']

In [ ]:
# Make a DataFrame that contains only the abovementioned columns, and name it as df_selected_business
df_selected_business = df_filtered[selected_features]

In [ ]:
# Rename the column name "stars" to "avg_stars" to avoid naming conflicts with review dataset
df_selected_business = df_selected_business.rename(columns = {"stars": "ave_stars"})

In [ ]:
# Inspect your DataFrame
df_selected_business.head(2)

#### Save results to csv files

In [ ]:
# Save to ./data/selected_business.csv for your next task
df_selected_business.to_csv("./yelp_dataset_challenge_round9/selected_business.csv", 
                            encoding = 'utf-8', index = False)

In [ ]:
# Try reload the csv file to check if everything works 
df_business_cleaned = pd.read_csv("./yelp_dataset_challenge_round9/selected_business.csv")
df_business_cleaned.head(2)

In [ ]:
df_business_cleaned.info()

### Use the "business_id" column to filter review data

* We want to make a DataFrame that contain and only contain the reviews about the business entities we just obtained

#### Load review dataset

In [ ]:
'''
The following code takes too much memory to run on my computer. 
I get the list of json files first, then convert it into dataframe
'''
with open(file_review) as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
df_review.head(2)


In [6]:
time0 = time()
reviews = set([])
with open(file_review) as f:
    for line in f:
        reviews.add(json.loads(line))
timet = time()
print "time used:", timet-time0, "seconds"

TypeError: unhashable type: 'dict'

In [ ]:
df_review = pd.DataFrame(reviews)
df_review.head(2)

In [ ]:
df_review.info()

#### Prepare dataframes to be joined, - on business_id

In [ ]:
# Prepare the business dataframe and set index to column "business_id", and name it as df_left
df_left = df_business_cleaned.set_index("business_id")
df_left.head(2)

In [ ]:
# Prepare the review dataframe and set index to column "business_id", and name it as df_right
df_right = df_review.set_index("business_id")
df_right.head(2)

#### Join! and reset index

In [ ]:
# Join df_left and df_right. What type of join? #use inner join
df_join = pd.merge(df_left, df_right, left_index = True, 
                   right_index = True, how = "inner")
df_join.head(2)

In [ ]:
# You may want to reset the index 
df_join = df_join.reset_index()
df_join.head(2)

In [ ]:
df_join.info()

#### We further filter data by date, e.g. keep comments from last 2 years

* Otherwise your laptop may crush on memory when running machine learning algorithms
* Purposefully ignoring the reviews made too long time ago

In [ ]:
# Make a filter that selects date after 2015-01-20
last2year = pd.to_datetime(df_join["date"]) >= pd.to_datetime("2016-06-16", format = "%Y-%m-%d")

In [ ]:
# Filter the joined DataFrame and name it as df_final
df_join = df_join[last2year]
df_join.info()


In [ ]:
df_join.head(2)

#### Take a glance at the final dataset

* Do more EDA here as you like!

In [ ]:
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
# calculate counts of reviews per business entity, and plot it

review_per_business = df_join["business_id"].value_counts()

review_per_business.plot.hist(bins = 100, 
                              title = "Number of reviews per business",
                             alpha = 0.5)

Most of business have less than 200 reviews

In [ ]:
# tranform the count to log(count) to better view the distribution
review_per_business_log = np.log(df_join["business_id"].value_counts())

ax = review_per_business_log.plot.hist(bins = 50, 
                              title = "Number of reviews per business",
                             alpha = 0.5)
ax.set_xlabel("log(count)")

Some business have more than 100, even 10000 reviews

In [ ]:
# plot the distribution of the number of reviews from users
review_per_user_log = np.log(df_join["user_id"].value_counts())
review_per_user_log.plot.hist(bins = 10, alpha = 0.5)

Most users have less than 100 reviews

In [ ]:
# rating stars distribution
stars = df_join["stars"].value_counts()
stars.plot.bar(alpha = 0.5)

In [ ]:
df_join["text_len"] = df_join["text"].apply(lambda x: len(list(x.split())))

In [ ]:
df_join.boxplot(by = "stars", column = "text_len")

It looks like the five star reviews are usually shorter

## Save your preprocessed dataset to csv file

* Respect your laptop's hard work! You don't want to make it run everything again.

In [ ]:
# Save to ./data/last_2_years_restaurant_reviews.csv for your next task
df_join.to_csv("./yelp_dataset_challenge_round9/last_2_years_restaurant_reviews.csv", 
               encoding = 'utf-8', index = False, date_format = True)
print "writing done"

In [ ]:
# reload data to check
last_2_years_restaurant = pd.read_csv("./yelp_dataset_challenge_round9/last_2_years_restaurant_reviews.csv")
last_2_years_restaurant.info()
last_2_years_restaurant.head(2)